
# ReAct + Reflection Agent Demo (LangGraph)

This notebook demonstrates the **core cognition loop** behind agentic AI:

> **Thought → Action → Observation → Reflection → Improved Thought**

We will:
- Build a minimal ReAct-style agent in **LangGraph**
- Add a simple **Reflection step** that critiques mistakes
- Compare answers *with vs without* reflection on the same query



## 1. Setup

We use:

- `langgraph` for graph-based orchestration  
- `langchain-openai` for LLM calls  
- A tiny in-notebook tool (a mock knowledge base)  


In [ ]:

%pip install -q langgraph langchain-openai langchain mcp  # mcp not strictly needed here but part of your stack

import os
os.environ.setdefault("OPENAI_API_KEY", "sk-REPLACE_ME")



## 2. A Tiny World & Tools

We define a simple **knowledge base** as a Python dict and expose it as a tool.

The goal:
- Let the agent **think** about *what* to look up
- Call the tool
- Reflect if it missed anything important


In [ ]:

from typing import Dict
from langchain.tools import tool

KB: Dict[str, str] = {
    "MCP": "Model Context Protocol is a standard for exposing tools and data sources to LLM-based agents.",
    "LangGraph": "LangGraph lets you model LLM applications as graphs of nodes, edges, and state.",
    "ReAct": "ReAct is a reasoning pattern where the LLM alternates between thoughts and actions.",
}

@tool
def lookup_concept(name: str) -> str:
    """Look up a short definition for a concept like MCP, LangGraph, or ReAct."""
    return KB.get(name, f"No entry found for {name}.")



## 3. Building a ReAct + Reflect Graph

We will create:

- `llm_node`: produces thoughts & (optionally) tool calls  
- `tool_node`: executes tools (`lookup_concept`)  
- `reflect_node`: gets the draft answer and tries to improve it  


In [ ]:

from langgraph.graph import StateGraph, START, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

tools = [lookup_concept]
tool_node = ToolNode(tools)

def llm_node(state: MessagesState):
    bound = llm.bind_tools(tools)
    response = bound.invoke(state["messages"])
    return {"messages": state["messages"] + [response]}



### Reflection Node

After the LLM uses tools and generates an answer, we can ask it to:

- Review its own answer
- Check for missing pieces or errors
- Produce an **improved** final response


In [ ]:

def reflection_node(state: MessagesState):
    messages = state["messages"]
    # Take the last assistant message as the draft
    draft = messages[-1]["content"] if isinstance(messages[-1]["content"], str) else str(messages[-1]["content"])
    prompt = (
        "You are a careful reviewer. The following is your OWN draft answer. "
        "Identify mistakes, missing key ideas, and then produce a refined final answer.\n\n"
        f"Draft answer:\n{draft}"
    )
    review = llm.invoke([{"role": "user", "content": prompt}])
    return {"messages": messages + [review]}



### Assemble the Graph

We want:

```text
START → llm → (tools?) → llm → reflection → END
```


In [ ]:

from langgraph.graph import END

builder = StateGraph(MessagesState)
builder.add_node("llm", llm_node)
builder.add_node("tools", tool_node)
builder.add_node("reflect", reflection_node)

builder.add_edge(START, "llm")
builder.add_conditional_edges("llm", tools_condition, default_edge="reflect")
builder.add_edge("tools", "llm")
builder.add_edge("reflect", END)

react_reflect_graph = builder.compile()
react_reflect_graph



## 4. Run an Example Query

We’ll ask something that *requires* combining MCP, LangGraph, and ReAct:

> *“Explain how MCP, LangGraph, and ReAct work together to build a powerful agentic AI system.”*


In [ ]:

from pprint import pprint

question = (
    "Explain how MCP, LangGraph, and ReAct can work together to build a powerful agentic AI system. "
    "Use the available lookup tool if needed."
)

state = {"messages": [{"role": "user", "content": question}]}
result = react_reflect_graph.invoke(state)

print("=== Final messages ===")
for i, m in enumerate(result["messages"]):
    print(f"\n[{i}] role={m['role']}")
    pprint(m["content"])



## 5. Takeaways

- **ReAct** provides a **thinking pattern**: alternate between thoughts and actions.  
- **LangGraph** gives a **programmable control structure** around that pattern.  
- **Reflection** layers **self-critique** on top, improving reliability and depth.

You can now adapt this graph to:
- Add more tools (RAG, APIs, databases)  
- Add stronger reflection (compare multiple candidate answers)  
- Add memory for cross-session learning  
